In [1]:
import insightface

In [2]:
face_model = insightface.app.FaceAnalysis()
face_model.prepare(ctx_id=-1, det_size=(640, 640))


/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

In [3]:
from imutils import paths, resize
import cv2
import os

In [4]:
from tqdm import tqdm

In [5]:
id_mapping = {}
name_mapping = ""

In [11]:
from annoy import AnnoyIndex

f = 512  # num of dimensions
annoy = AnnoyIndex(f, 'angular')

In [28]:
DETECT_THRESHOLD = 0.6

def embedding_extract(imgs_folder, id2name):
    global id_mapping, name_mapping
    
    id_mapping = {}
    name_mapping = id2name
    count = 0

    for img_path in paths.list_images(imgs_folder):
        try:
            dossier_id = img_path.split(os.path.sep)[-2]
            print(f"[DEBG] dossier_id: {dossier_id}")
            img = cv2.imread(img_path)
            # TODO resize
            img = resize(img, width=250)
            faces = face_model.get(img)
            if len(faces) == 1:
                for face in faces:
                    if face.det_score < DETECT_THRESHOLD:
                        continue
                    embedding = face.embedding.flatten()
                    print(f"[DEBUG] embedding size: {embedding.shape}")
                    annoy.add_item(count, embedding)
                    id_mapping[str(count)] = dossier_id
                    count += 1
        except Exception as e:
            continue

    annoy.build(4)
    print("[DEBUG] extract successful")

In [29]:
embedding_extract('lesserafim', 'lesserafim')

[DEBG] dossier_id: sakura


/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


[DEBUG] embedding size: (512,)
[DEBG] dossier_id: sakura
[DEBG] dossier_id: chaewon
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: eunchae
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: yunjin
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: kazuha
[DEBUG] embedding size: (512,)
[DEBUG] extract successful


In [30]:
print(id_mapping)
print(name_mapping)

{'0': 'sakura', '1': 'chaewon', '2': 'eunchae', '3': 'yunjin', '4': 'kazuha'}
lesserafim


In [52]:
import time
import numpy as np
from vidgear.gears import CamGear
import uuid

def analyze(filename, sub_dir, video_id):
    """ Analyze video

    Args:
        video_path  : video full path
        sub_dir     : where to save dossier images found
        video_id    : ID of video

    Returns:
        results     : List<(dossier_id/name, image_path)>
    """
    print(f'[DEBUG] id_mapping: {id_mapping}')
    results = []
    unknown_faces = []

    milestone = 5
    extra = 5

    status = 'PROCESSING'
    # before_time = time.time()

    image = cv2.imread(filename)
    height, width, _ = image.shape
    frame_rate = 60
    frame_id = 0
    start_time = time.time()
    completed_frames = 0

    try:
        completed_frames += 1
        small_frame = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)
        faces = face_model.get(small_frame)
        
        if len(faces) > 0:
            timestamp = time.time() - start_time
            print(f"[INFO] found {len(faces)} faces in {timestamp}")
                    
        print([x.det_score for x in faces])
        for face in faces:
            print(f" >> det_score: {face.det_score}")
            if face.det_score < 0.6:
                continue
            print(f" >> found face with p: {face.det_score}")

            embedding = face.embedding.flatten()
            closest_indices, distances = annoy.get_nns_by_vector(
                        embedding, n=1, include_distances=True)
            closest_embedding = annoy.get_item_vector(
                closest_indices[0])
            print(closest_indices)
            similarity = (2. - distances[0] ** 2) / 2.
            print(f" >> sim: {similarity}")

            left, top, right, bottom = tuple(
                        face.bbox.astype(np.int).flatten())
            if left < 0:
                left = 1
            if top < 0:
                top = 1
            if right > width:
                right = width - 1
            if bottom > height:
                bottom = height - 1
            face_size = (right - left) * (bottom - top)
            print(left, top, right, bottom)

            if similarity >= 0.2:
                print(f" >> guess: {id_mapping[str(closest_indices[0])]}")
                crop_img = image[top:bottom, left:right]
                crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2BGR)

                dossier_id = id_mapping[str(closest_indices[0])]
                filename = dossier_id + '_' + \
                    str(uuid.uuid4()) + '.jpg'
                path_to_save = os.path.join(
                    sub_dir, filename)
                cv2.imwrite(path_to_save, crop_img)
                cv2.imshow(path_to_save, crop_img)
                results.append(
                    (dossier_id, path_to_save))
    finally:
        cv2.destroyAllWindows() 
        print("[INFO] finished video analysing")
    return results, status

In [53]:
analyze('input_2.jpg', 'dossier', 'lesserafim')

[DEBUG] id_mapping: {'0': 'sakura', '1': 'chaewon', '2': 'eunchae', '3': 'yunjin', '4': 'kazuha'}


/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


[INFO] found 5 faces in 1.3276469707489014
[0.903146, 0.8850911, 0.87207186, 0.8649545, 0.809651]
 >> det_score: 0.9031460285186768
 >> found face with p: 0.9031460285186768
[4]
 >> sim: 0.2937805731688101
39 17 57 39
 >> guess: kazuha
 >> det_score: 0.8850911259651184
 >> found face with p: 0.8850911259651184
[0]
 >> sim: 0.3221736485199642
102 21 119 44
 >> guess: sakura
 >> det_score: 0.8720718622207642
 >> found face with p: 0.8720718622207642
[2]
 >> sim: 0.42144804319011087
172 21 189 44
 >> guess: eunchae
 >> det_score: 0.8649544715881348
 >> found face with p: 0.8649544715881348
[3]
 >> sim: 0.34350944620461377
132 17 149 39
 >> guess: yunjin
 >> det_score: 0.8096510171890259
 >> found face with p: 0.8096510171890259
[1]
 >> sim: 0.29182409732987225
70 25 87 44
 >> guess: chaewon
[INFO] finished video analysing


/var/folders/_x/zckfnlk92vq72rvw9szdlzqw0000gn/T/ipykernel_31721/1539852019.py:60: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  face.bbox.astype(np.int).flatten())


([('kazuha', 'dossier/kazuha_9565056d-ad22-481d-80d9-8711bfd538da.jpg'),
  ('sakura', 'dossier/sakura_d7ea3131-2750-4c73-a035-3ad8749c9a9e.jpg'),
  ('eunchae', 'dossier/eunchae_34dd6130-ee19-4d17-8e28-035374eadd13.jpg'),
  ('yunjin', 'dossier/yunjin_ec2b0b9d-5acd-4ba4-8fd9-09c005aea23c.jpg'),
  ('chaewon', 'dossier/chaewon_bbb05f43-1cc3-4746-b7bf-ac09e5fe2852.jpg')],
 'PROCESSING')

: 